# Importing the libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extracting the dataset

In [2]:
df=pd.read_csv("fer2013.csv")
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
df.dtypes

emotion     int64
pixels     object
Usage      object
dtype: object

In [4]:
#Code cell to seperate out training and testing data
X_train=[]
y_train=[]
X_test=[]
y_test=[]
for index,row in df.iterrows():
    p=row['pixels'].split(" ")
    if row['Usage']=='Training':
        X_train.append(np.array(p))
        y_train.append(row['emotion'])
    elif row['Usage']=='PublicTest':
        X_test.append(np.array(p))
        y_test.append(row['emotion'])

In [5]:
#Conversion from object data type to integer
X_train=np.array(X_train,dtype='uint8')
y_train=np.array(y_train,dtype='uint8')
X_test=np.array(X_test,dtype='uint8')
y_test=np.array(y_test,dtype='uint8')

In [6]:
#reshaping of images
X_train=X_train.reshape(X_train.shape[0],48,48,1)
X_test=X_test.reshape(X_test.shape[0],48,48,1)

In [7]:
#rescaling and formation of multiple images
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=0.2,
                                 zoom_range=0.2,horizontal_flip=True,
                                 rotation_range=5)
test_datagen=ImageDataGenerator(rescale=1/255)

In [8]:
#optimising data for efficient implementation of model and forming batches
train_flow=train_datagen.flow(X_train,y_train,batch_size=64)
test_flow=test_datagen.flow(X_test,y_test,batch_size=64)

# Network

In [9]:
cnn=tf.keras.models.Sequential()

Convolution

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=[48,48,1]))

Pooling

In [11]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Convolution

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))

Pooling

In [13]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Convolution

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))

Pooling

In [15]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Flattening

In [16]:
cnn.add(tf.keras.layers.Flatten())

Full connection

In [17]:
cnn.add(tf.keras.layers.Dense(units=64,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=256,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=512,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1024,activation='relu'))

Output Layer

In [18]:
cnn.add(tf.keras.layers.Dense(units=7,activation='softmax'))

Fitting of Model to data

In [19]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=cnn.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train)/64, 
                    epochs=70,  
                    verbose=1,  
                    validation_data=test_flow,
                    validation_steps=len(X_test)/64)

C:\Users\rosha\AppData\Local\Temp\ipykernel_17312\2649564826.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=cnn.fit_generator(train_flow,


Epoch 1/70
448/448 [==============================] - 67s 146ms/step - loss: 1.7687 - accuracy: 0.2722 - val_loss: 1.6249 - val_accuracy: 0.3650
Epoch 2/70
448/448 [==============================] - 61s 135ms/step - loss: 1.5903 - accuracy: 0.3711 - val_loss: 1.4893 - val_accuracy: 0.4224
Epoch 3/70
448/448 [==============================] - 64s 142ms/step - loss: 1.4938 - accuracy: 0.4159 - val_loss: 1.4260 - val_accuracy: 0.4372
Epoch 4/70
448/448 [==============================] - 71s 158ms/step - loss: 1.4360 - accuracy: 0.4419 - val_loss: 1.3909 - val_accuracy: 0.4625
Epoch 5/70
448/448 [==============================] - 72s 161ms/step - loss: 1.4035 - accuracy: 0.4574 - val_loss: 1.3819 - val_accuracy: 0.4795
Epoch 6/70
448/448 [==============================] - 71s 159ms/step - loss: 1.3561 - accuracy: 0.4740 - val_loss: 1.3509 - val_accuracy: 0.4851
Epoch 7/70
448/448 [==============================] - 70s 157ms/step - loss: 1.3331 - accuracy: 0.4845 - val_loss: 1.2879 - val_ac

In [20]:
#save model
cnn.save('CNN model')

INFO:tensorflow:Assets written to: CNN model\assets


INFO:tensorflow:Assets written to: CNN model\assets


In [2]:
#load the model
cnn_copy=tf.keras.models.load_model('CNN model')

# Real-Time Emotion Detection using Webcam

In [4]:
import cv2
face_haar_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

In [5]:
frame=cv2.VideoCapture(0)

while True:
    _,test_image=frame.read()
    converted_image=cv2.cvtColor(test_image,cv2.COLOR_BGR2GRAY)                # Conversion to Black and White of a Frame

    faces=face_haar_cascade.detectMultiScale(converted_image,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(test_image,(x,y),(x+w,y+h),(255,0,0))                    # Detecting Face in Camera
        roi_gray=converted_image[y:y+w,x:x+h]                                  # Marking region of interest for emotion recognition
        roi_gray=cv2.resize(roi_gray,(48,48))
        image_pixels=np.asarray(roi_gray)                                      # Conversion of image to array of pixels
        image_pixels=np.expand_dims(image_pixels,axis=0)
        image_pixels=image_pixels/255
        image_pixels=image_pixels.reshape(1,48,48,1)                           # Reshaping image_pixels to match input dimensions of the model
        predictions=cnn_copy.predict(image_pixels)
        
        max_index=np.argmax(predictions[0])                                    # Finding index of Emotion with maximum Probability
        emotion_detection=('angry','disgust','fear','happy','sad','surprise','neutral')
        emotion_prediction=emotion_detection[max_index]

        cv2.putText(test_image,emotion_prediction,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,2,(128,255,0),3)  # Printing Emotion detected on the frame
        cv2.resize(test_image,(480,340))
        cv2.imshow('Emotion',test_image)

    if cv2.waitKey(1) & 0xFF==ord('q'):                                        # Exit from Window
        break


frame.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 17ms/step


In [34]:
test_image=cv2.imread(r"C:\Users\rosha\Downloads\Emotion Recognition\w.jpg")
converted_image=cv2.cvtColor(test_image,cv2.COLOR_BGR2GRAY)                # Conversion to Black and White of a Frame
faces=face_haar_cascade.detectMultiScale(converted_image,2,2)
print(faces)
for (x,y,w,h) in faces:
    cv2.rectangle(test_image,(x,y),(x+w,y+h),(255,0,0))                    # Detecting Face in Camera
    roi_gray=converted_image[y:y+w,x:x+h]                                  # Marking region of interest for emotion recognition
    roi_gray=cv2.resize(roi_gray,(48,48))
    image_pixels=np.asarray(roi_gray)                                      # Conversion of image to array of pixels
    image_pixels=np.expand_dims(image_pixels,axis=0)
    image_pixels=image_pixels/255
    image_pixels=image_pixels.reshape(1,48,48,1)                           # Reshaping image_pixels to match input dimensions of the model
    predictions=cnn_copy.predict(image_pixels)

    max_index=np.argmax(predictions[0])                                    # Finding index of Emotion with maximum Probability
    emotion_detection=('angry','disgust','fear','happy','sad','surprise','neutral')
    emotion_prediction=emotion_detection[max_index]

    cv2.putText(test_image,emotion_prediction,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,2,(128,255,0),3)  # Printing Emotion detected on the frame
    cv2.resize(test_image,(480,340))
    cv2.imshow('Emotion',test_image)

if cv2.waitKey(0) & 0xFF==ord('q'):                                        # Exit from Window
    cv2.destroyAllWindows()
    
cv2.destroyAllWindows()

[[168  69 384 384]]
1/1 [==============================] - 0s 16ms/step
